In [176]:
import kagglehub
import os 
import pandas as pd
import numpy as np

# Download latest version
path = kagglehub.dataset_download("heesoo37/120-years-of-olympic-history-athletes-and-results")

csv_path = f"{path}/athlete_events.csv"

df = pd.read_csv(csv_path , sep=',' ,encoding="ISO-8859-1")

df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## Data Cleaning

In [177]:
df.shape

(271116, 15)

In [178]:
df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [179]:
df.describe()

,ID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


In [180]:
missing_val_percentage = (df.isnull().sum()/len(df) )*100
missing_val_percentage.sort_values(ascending=False)

Medal     85.326207
Weight    23.191180
Height    22.193821
Age        3.494445
Sex        0.000000
ID         0.000000
Name       0.000000
Team       0.000000
NOC        0.000000
Year       0.000000
Games      0.000000
Season     0.000000
City       0.000000
Sport      0.000000
Event      0.000000
dtype: float64

In [181]:
df_copy = df.copy()
df_copy['Medal'] = df_copy['Medal'].fillna('No Medal')
cols = ['Age' , 'Weight' , 'Height']
for col in cols:
    df_copy[col] = df_copy[col].fillna(df.groupby('Sport')[col].transform('median'))
    df_copy[col] = df_copy[col].fillna(df.groupby('Sex')[col].transform('median'))
    
print(df_copy[df_copy['Weight'].isnull()]['Sport'].unique())

[]


In [182]:
from sklearn.preprocessing import OneHotEncoder

In [183]:
df_copy.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal


In [184]:
# one hot encode medal colums
encoded = pd.get_dummies(df_copy['Medal'], dtype=int)
df_copy = pd.concat([df_copy , encoded] , axis=1)
df_copy.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,0,0,1,0


In [185]:
df_copy.sample(10)
df_copy.drop(columns=['ID'] , inplace=True)

In [186]:
# remove whitespaces
for col in df_copy.columns:
    df_copy.replace(" " , "")
df_copy.sample(10)

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
239001,Kristian James Thomas,M,27.0,180.0,79.0,Great Britain,GBR,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,No Medal,0,0,1,0
190030,"Lawrence ""Larry"" Plenert",M,23.0,200.0,88.0,Canada,CAN,1976 Summer,1976,Summer,Montreal,Volleyball,Volleyball Men's Volleyball,No Medal,0,0,1,0
128684,Galina Alekseyevna Kukleva,F,29.0,176.0,66.0,Russia,RUS,2002 Winter,2002,Winter,Salt Lake City,Biathlon,Biathlon Women's 4 x 7.5 kilometres Relay,Bronze,1,0,0,0
187438,Paulo Roberto Petterle,M,27.0,197.0,94.0,Brazil,BRA,1976 Summer,1976,Summer,Montreal,Volleyball,Volleyball Men's Volleyball,No Medal,0,0,1,0
106994,Vincent Nicolas Jay,M,24.0,182.0,73.0,France,FRA,2010 Winter,2010,Winter,Vancouver,Biathlon,Biathlon Men's 4 x 7.5 kilometres Relay,No Medal,0,0,1,0
181014,Nontapat Panchan,M,26.0,176.0,70.0,Thailand,THA,2008 Summer,2008,Summer,Beijing,Fencing,"Fencing Men's Foil, Individual",No Medal,0,0,1,0
228980,Lucy Stephan,F,24.0,174.0,66.0,Australia,AUS,2016 Summer,2016,Summer,Rio de Janeiro,Rowing,Rowing Women's Coxed Eights,No Medal,0,0,1,0
11085,Anbal Guillermo Avalos,M,25.0,176.0,67.0,Argentina,ARG,1948 Summer,1948,Summer,London,Athletics,Athletics Men's 4 x 400 metres Relay,No Medal,0,0,1,0
194956,Andr Queill,M,20.0,172.0,63.0,France,FRA,1952 Summer,1952,Summer,Helsinki,Boxing,Boxing Men's Light-Middleweight,No Medal,0,0,1,0
135017,Lee Hui-Cheng,F,16.0,166.0,68.0,Chinese Taipei,TPE,1984 Summer,1984,Summer,Los Angeles,Athletics,Athletics Women's Javelin Throw,No Medal,0,0,1,0


In [187]:
duplicate_val_cols = ['Team' , 'Event' , 'Medal' , 'Year' , 'City' ,'Sport' , 'Games'] 
df_copy.drop_duplicates(subset=duplicate_val_cols , inplace=True)
df_copy.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,0,0,1,0


In [188]:
# we will only analyze by summer olympics
df_copy = df_copy[df_copy['Season']=='Summer']
df_copy.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
26,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,67.0,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,No Medal,0,0,1,0


In [189]:
df_copy.isnull().sum()

Name        0
Sex         0
Age         0
Height      0
Weight      0
Team        0
NOC         0
Games       0
Year        0
Season      0
City        0
Sport       0
Event       0
Medal       0
Bronze      0
Gold        0
No Medal    0
Silver      0
dtype: int64

In [190]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105600 entries, 0 to 271099
Data columns (total 18 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Name      105600 non-null  object 
 1   Sex       105600 non-null  object 
 2   Age       105600 non-null  float64
 3   Height    105600 non-null  float64
 4   Weight    105600 non-null  float64
 5   Team      105600 non-null  object 
 6   NOC       105600 non-null  object 
 7   Games     105600 non-null  object 
 8   Year      105600 non-null  int64  
 9   Season    105600 non-null  object 
 10  City      105600 non-null  object 
 11  Sport     105600 non-null  object 
 12  Event     105600 non-null  object 
 13  Medal     105600 non-null  object 
 14  Bronze    105600 non-null  int64  
 15  Gold      105600 non-null  int64  
 16  No Medal  105600 non-null  int64  
 17  Silver    105600 non-null  int64  
dtypes: float64(3), int64(5), object(10)
memory usage: 15.3+ MB


## EDA

## Total Medals By each Country

In [209]:
# get medals by each country
medal_tally = df_copy.groupby(['NOC' , 'Team']).sum()[['Gold' , 'Silver' , 'Bronze']].sort_values(by='Gold' , ascending=False).reset_index()
medal_tally['Total'] = medal_tally['Gold'] + medal_tally['Silver'] + medal_tally['Bronze']
medal_tally

,NOC,Team,Gold,Silver,Bronze,Total
0,USA,United States,997,780,682,2459
1,URS,Soviet Union,393,317,294,1004
2,GBR,Great Britain,243,286,276,805
3,GER,Germany,224,256,273,753
4,FRA,France,221,232,260,713
...,...,...,...,...,...,...
1168,USA,United States-14,0,0,0,0
1169,CAN,Beaver,0,0,0,0
1170,USA,United States-3,0,0,1,1
1171,USA,United States-4,0,0,1,1


## Year wise Comparision of Medals by each Country

In [207]:
year_wise_df = df_copy.groupby(['Team', 'Year']).sum()[['Gold', 'Silver', 'Bronze']].sort_values(by=['Year' , 'Gold'],ascending=False).reset_index()
year_wise_df['Total'] = year_wise_df['Gold'] + year_wise_df['Silver'] + year_wise_df['Bronze']
year_wise_df

,Team,Year,Gold,Silver,Bronze,Total
0,United States,2016,45,36,36,117
1,Great Britain,2016,27,23,17,67
2,China,2016,25,18,25,68
3,Russia,2016,18,17,20,55
4,Germany,2016,16,10,15,41
...,...,...,...,...,...,...
4109,Greece-1,1896,0,1,0,1
4110,Greece-2,1896,0,0,0,0
4111,Greece-3,1896,0,0,0,0
4112,Italy,1896,0,0,0,0


In [ ]:
'''
Todo:
1) Do Overall analysis of Country 
2) Make 
'''